<a href="https://colab.research.google.com/github/riccardomarin/GeoHumanUNIVR/blob/main/UNIVR_Visualizations_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will play with visualizations and representations using some common python libraries. \
Two fundamental libraries are Open3D: https://www.open3d.org/ \
And trimesh: https://trimesh.org/ \

In [ ]:
## Useful 3D libraries not pre-installed in colab
!pip install open3d
!pip install trimesh

We will also need some 3D data.

In [ ]:
## Download 3D data
!git clone https://github.com/riccardomarin/CDG.git
%cd CDG
!git checkout iss53
%cd ..
!cp -r ./CDG/* .


# Getting a mesh
import os
os.system('wget https://github.com/czh-98/REALY/raw/master/data/3DDFA_v2.obj')

Imports and utility functions

In [ ]:
%matplotlib inline

## Imports
import open3d as o3d
import os
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import skimage
import scipy
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
from trimesh.visual import texture, TextureVisuals
from trimesh import Trimesh
from PIL import Image
import torch


## Utility functions to convert numpy arrays into Open3D types
v3f = o3d.utility.Vector3dVector
v3i = o3d.utility.Vector3iVector

### Utility visualization function
def plot_mesh(verts, trivs, colors=None, colorscale=[[0, 'rgb(0,0,255)'], [0.5, 'rgb(255,255,255)'], [1, 'rgb(255,0,0)']], point_size=2, wireframe=False):
    "Draw multiple triangle meshes side by side"
    "colors must be list(range(num colors))"
    if type(verts) is not list:
        verts = [verts]
    if type(trivs) is not list:
        trivs = [trivs]
    if type(colors) is not list:
        colors = [colors]
    if type(verts[0]) == torch.Tensor:
        to_np = lambda x: x.detach().cpu().numpy()
        verts=[ to_np(v) for v in verts]


    "Check device for torch tensors"
    def to_cpu(v):
        if torch.is_tensor(v):
            return v.data.cpu()
        return v;
    verts = [to_cpu(x) for x in verts]
    trivs = [to_cpu(x) for x in trivs]
    colors = [to_cpu(x) for x in colors]


    nshapes = min([len(verts), len(colors), len(trivs)])

    fig = make_subplots(rows=1, cols=nshapes, specs=[[{'type': 'surface'} for i in range(nshapes)]])

    for i, [vert, triv, col] in enumerate(zip(verts, trivs, colors)):
        if triv is not None:
            if col is not None:
                mesh = go.Mesh3d(x=vert[:, 0], z=vert[:, 1], y=vert[:, 2],
                                 i=triv[:, 0], j=triv[:, 1], k=triv[:, 2],
                                 intensity=col,
                                 colorscale=colorscale,
                                 color='lightpink', opacity=1)
            else:
                mesh = go.Mesh3d(x=vert[:, 0], z=vert[:, 1], y=vert[:, 2],
                                 i=triv[:, 0], j=triv[:, 1], k=triv[:, 2])
        else:
            if col is not None:
                mesh = go.Scatter3d(x=vert[:, 0], z=vert[:, 1], y=vert[:, 2],
                                    mode='markers',
                                    marker=dict(
                                        size=point_size,
                                        color=col,  # set color to an array/list of desired values
                                        colorscale='Viridis',
                                        # choose a colorscale
                                        opacity=1
                                    ))
            else:
                mesh = go.Scatter3d(x=vert[:, 0], z=vert[:, 1], y=vert[:, 2],
                                    mode='markers',
                                    marker=dict(
                                        size=point_size,  # set color to an array/list of desired values
                                        colorscale='Viridis',  # choose a colorscale
                                        opacity=1
                                    ))

        fig.add_trace(mesh, row=1, col=i + 1)
        fig.get_subplot(1, i + 1).aspectmode = "data"

        if wireframe and triv is not None:
            tripts = vert[triv]
            tripts = np.concatenate([tripts,tripts[:,0:1,:],tripts[:,0:1,:]*np.nan],1).reshape(-1,3)
            lines = go.Scatter3d(
                   x=tripts[:,0],
                   y=tripts[:,2],
                   z=tripts[:,1],
                   mode='lines',
                   name='',
                   line=dict(color= '#000000', width=2))
            fig.add_trace(lines, row=1, col=i + 1)

        camera = dict(
            up=dict(x=0, y=0, z=1),
            center=dict(x=0, y=0, z=0),
            eye=dict(x=0, y=4, z=-1)
        )
        fig.get_subplot(1, i + 1).camera = camera

    #     fig = go.Figure(data=[mesh], layout=layout)
    fig.update_layout(
        #       autosize=True,
        margin=dict(l=10, r=10, t=2, b=2))
        #paper_bgcolor="LightSteelBlue")
    #fig.show()
    return fig


## Load and visualize triangle meshes

Method 1. Open3D is quick -- but on Colab rather limited (it is still an experimental feature)
http://www.open3d.org/docs/release/python_api/open3d.visualization.draw_plotly.html

In [ ]:
#@title {vertical-output: true}

M = o3d.io.read_triangle_mesh('3DDFA_v2.obj')

# I define a color for the vertices but the visualization does not support colors
colors = np.asarray(M.vertices) * 0

# Notice that open3d use its own data types.
M.vertex_colors = v3f(colors)

o3d.visualization.draw_plotly([M])

This Open3D visualization runs on plotly which is a quite powerful library: https://plotly.com/ \
 We can directly use it to visualize a point cloud

In [ ]:
#@title {vertical-output: true}

## Defining the points
points = np.asarray(M.vertices)

## Defining the colors
colors = (points - np.min(points, axis=0))/(np.max(points, axis=0) - np.min(points, axis=0))

#colors[:,[0,1]] = 0

# Define a figure
fig = go.Figure(
  data=[

      # Define the data in the figure -- We have a 3D point Cloud
    go.Scatter3d(
      x=points[:,0], y=points[:,1], z=points[:,2],
      mode='markers',

      # Points parameters: their size and their colors
      marker=dict(size=1, color=colors)
)
]

# Uncomment these to remove the axes
  ,
  layout=dict(
    scene=dict(
      xaxis=dict(visible=False),
      yaxis=dict(visible=False),
      zaxis=dict(visible=False)
)
)
)
fig.show()


Another popular library is trimesh:
https://trimesh.org/

In [ ]:
#@title {vertical-output: true}
import trimesh

# Defining faces and vertices
v = np.asarray(points)
f = np.asarray(M.triangles)
tri = trimesh.Trimesh(v,f)

s = trimesh.Scene()
s.add_geometry(trimesh.creation.axis())
s.add_geometry(tri)
s.show()

# It gives also an example of Back-face culling
# https://en.wikipedia.org/wiki/Back-face_culling

# Computing Mesh statistics

Open3D TriangleMesh object has two main attributes:
1. Vertices
2. Triangles

but once you set them, you can also ask to obtain more property.

In [ ]:
#@title {vertical-output: true}
M = o3d.io.read_triangle_mesh('3DDFA_v2.obj')

print(M)
print('Vertices:')
print(np.asarray(M.vertices))
print('Triangles:')
print(np.asarray(M.triangles))

self_intersecting = M.is_self_intersecting() # returns True if there exists a triangle in the mesh that is intersecting another mesh.
watertight = M.is_watertight()               # A watertight mesh can be seen as a unique, closed surface
orientable = M.is_orientable()               # the triangles can be oriented in such a way that all normals point towards the outside.

print(f"  self_intersecting:      {self_intersecting}")
print(f"  watertight:             {watertight}")
print(f"  orientable:             {orientable}")

# An example of non-orientable surface: Moebius Strip
# https://www.researchgate.net/publication/354157936/figure/fig2/AS:1061263429881856@1630036327084/The-Moebius-strip-is-not-orientable.png

But it has many other fields that you can compute

http://www.open3d.org/docs/release/tutorial/geometry/mesh.html

# Synthetic camera view

Sometimes we want to generate images from our 3D data. A few interesting applications:\
1) Obtaining visual features (e.g., process them with DINO)\
2) Learning using 2D CNN\
3) Procedurally generating visualizations without manual intervention

We will use this mesh

In [ ]:
#@title {vertical-output: true}

# Let's load a different shape
M = o3d.io.read_triangle_mesh('./CDG/faust_ply/tr_reg_000.ply')
verts = np.asarray(M.vertices)
faces = np.asarray(M.triangles)


verts = np.asarray(M.vertices)  # Vertices
faces = np.asarray(M.triangles) # Triangles


plot_mesh(verts, faces)

Let's see how to do that in Open3D. Maybe it is not the most efficient way (I think pytorch3D has a much better and GPU-ready pipeline), but it is convenient for us.

In [ ]:
#@title {vertical-output: true}

# Open3D requires to create a scene that you want to render
scene = o3d.t.geometry.RaycastingScene()
mesh = o3d.t.geometry.TriangleMesh.from_legacy(M)
_ = scene.add_triangles(mesh)

# We create a pinhole camera
rays = scene.create_rays_pinhole(fov_deg=60,              #Field of view
                                 center=[-1.5,-1.5,-1.5], #Look towards
                                 eye=[1,1,1],             #The position of the camera (Look from)
                                 up=[0,0,1],              #The up vector - orienting the camera so object is not inverted
                                 width_px=1024,           #Resolution
                                 height_px=1024)

# Then we shoot the rays from our
ans = scene.cast_rays(rays)

plt.imshow(ans['t_hit'].numpy())
plt.show()

We can also decorate the visualization with the normals (and a few other quantities)

I encourage you to load different shapes and change visualization parameters from above (e.g., remove the axes, visualize different functions, use plotly to show normals on the surface, change the pinhole camera perspective, ...)


## Processing geometry

Open3D has a full set of operations. For example, a common task is to estimate/reconstruct the underlying surface of a point cloud.

If we want to study the geometry, we often want to factor out the extrinsic orientation of the shape. Namely, processing geometry should be agnostic from its position in space. Hence, we often want to compute what we call "intrinsic quantities".

Note that this intrinsic quantitiy is fully computed using solely point cloud extrinsic information: no need for fancy and clean surface! We now verify that the curvature is invariant to the point cloud rotations

# Conversions between Representations

An intersting task is also to convert between different representations. For example, converting a mesh into a signed distance function (SDF). Let's visualize our mesh. I decided to visualize the wireframe (i.e., the edges of the triangles will be explicitly shown)

In [ ]:
#@title {vertical-output: true}
# We load a watertight mesh and we normalize it
M = o3d.io.read_triangle_mesh('./CDG/faust_ply/tr_reg_000.ply')
vertices = np.asarray(M.vertices)
vertices = (vertices - np.min(vertices))/(np.max(vertices) - np.min(vertices))
M.vertices = o3d.utility.Vector3dVector(vertices)
faces = np.asarray(M.triangles)

#@title {vertical-output: true}
# Subplot
fig = make_subplots(
          rows=1, cols=1,
          subplot_titles=(''),
          horizontal_spacing=0.02,
          specs=[[{"type": "scene"}]])

tri_vertices = vertices[faces]
Xe = []
Ye = []
Ze = []
for T in tri_vertices:
    Xe += [T[k%3][0] for k in range(4)] + [ None]
    Ye += [T[k%3][1] for k in range(4)] + [ None]
    Ze += [T[k%3][2] for k in range(4)] + [ None]

# Plot 1 - Just visualize the mesh grid
fig.add_trace(go.Scatter3d(x=Xe,
                     y=Ye,
                     z=Ze,
                     mode='lines',
                     name='',
                     line=dict(color= 'rgb(40,40,40)', width=0.5)), 1, 1);
# Camera and visualization layouts
fig.update_layout(width=1200, height=600, font_size=10)
fig.get_subplot(1, 1).aspectmode = "data"
fig.update_scenes(camera_eye_x=1.45, camera_eye_y=1.45, camera_eye_z=1.45);

fig.show()


Casting ray is a basic technique to check if a point is inside or outside. One can just check the intersections between a ray and the mesh triangles.

In [ ]:
#@title {vertical-output: true}

# o3d.t is a set of library tools that uses tensors
mesh = o3d.t.geometry.TriangleMesh.from_legacy(M)

# This provides basic ray casting functionality.
scene = o3d.t.geometry.RaycastingScene()

# Add our triangle mesh to the scene
_ = scene.add_triangles(mesh)

We can query any point in space and check if it is inside or outside, and how distant it is from the surface.

In [ ]:
#@title {vertical-output: true}

# Take the mesh central point
mean_vertices = np.mean(vertices,axis=0)[np.newaxis]

# And we use it as a query point
query_point = o3d.core.Tensor(mean_vertices, dtype=o3d.core.Dtype.Float32)

# Obtain unsigned distance
unsigned_distance = scene.compute_distance(query_point)

# Obtain signed distance
signed_distance = scene.compute_signed_distance(query_point)

# Is the point inside the shape?
occupancy = scene.compute_occupancy(query_point)

print("unsigned distance", unsigned_distance.numpy())
print("signed_distance", signed_distance.numpy())
print("occupancy", occupancy.numpy())

To visualize volumetric representation we need a volumentric visualization.

In [ ]:
# Set the centers
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
nc = 16

# Visualize a 16 x 16 x 16 grid
one = np.ones((nc, nc, nc))

# Grid creation
xc = np.arange(1, nc+1) - .5
yc = np.arange(1, nc+1) - .5
zc = np.arange(1, nc+1) - .5
xc_, yc_, zc_ = np.meshgrid(xc, yc, zc)
# ax.scatter(xc_.ravel(), yc_.ravel(), zc_.ravel(), marker = 10, c = 'r', s = 100) # Voxel Centers
ax.voxels(one, alpha = 0.12, edgecolor="k", shade=True) # Voxels
plt.tight_layout()
ax.set_title('Voxels and Voxel centers')
plt.show()

Here we visualize the occupancy, so every occupied cell will be solid, while the rest will be fully transparent.

In [ ]:
#@title {vertical-output: true}

##### Visualize Occupancy
query_coords = np.vstack((xc_.ravel(), yc_.ravel(), zc_.ravel())).T/nc
query_point = o3d.core.Tensor(query_coords, dtype=o3d.core.Dtype.Float32)

# Mark voxels that are occupied
signed_distance = scene.compute_signed_distance(query_point)
signed_distance = signed_distance.numpy().reshape((nc,nc,nc))

# Everything below a certain value is considered inside
threshold =  (np.cbrt(3) *  1/nc)/2

occ = signed_distance < threshold

fig = plt.figure()
ax = plt.gca()
ax = fig.add_subplot(projection='3d')
ax.voxels(occ, alpha = 0.9, edgecolor="k", shade=True) # Voxel visualization
ax.set_title('Occupancy')

ax.set_xlim([0, nc])
ax.set_ylim([0, nc])
ax.set_zlim([0, nc])
plt.show()

To visualize the SDF is a bit more complicated, as all the points in R^3 have a meaning. A strategy is to visualize just some level-sets. To do so, we extract the respective meshes

In [ ]:
ls = [0.01, 0.02, 0.05]

# Subplot
fig = make_subplots(
          rows=1, cols=len(ls),
          subplot_titles=(''),
          horizontal_spacing=0.02,
          specs=[[{"type": "scene"}]* len(ls)])

##### From SDF to mesh
for level_set, subp in zip(ls, np.arange(1,len(ls)+1)):
  vertices, faces, normals, _ = skimage.measure.marching_cubes(signed_distance, level=level_set)
  Mesh = o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(vertices),o3d.utility.Vector3iVector(faces))

  vertices = np.asarray(Mesh.vertices)
  faces = np.asarray(Mesh.triangles)
  x, y, z  = vertices.T
  i, j, k = faces.T

  tri_vertices = vertices[faces]
  Xe = []
  Ye = []
  Ze = []
  for T in tri_vertices:
      Xe += [T[k%3][0] for k in range(4)] + [ None]
      Ye += [T[k%3][1] for k in range(4)] + [ None]
      Ze += [T[k%3][2] for k in range(4)] + [ None]


  # Plot 1 - Just visualize the mesh grid
  lighting = dict(ambient=0.5,
                  diffuse=1,
                  fresnel=4,
                  specular=0.5,
                  roughness=0.05,
                  facenormalsepsilon=0)

  lightposition=dict(x=100,
                    y=100,
                    z=10000)

  fig.add_trace(go.Mesh3d(x=x, y=y, z=z,
                          i=i, j=j, k=k, colorscale='matter_r' ,
                          colorbar_len=0.85,
                          colorbar_x=0.625,
                          intensity=z, intensitymode='vertex',
                          flatshading=True), 1, subp);

  # Camera and visualization layouts
  fig.update_layout(width=1200, height=600, font_size=10)
  fig.update_traces(showscale=False)
  fig.get_subplot(1, int(subp)).aspectmode = "data"
fig.show()
